In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import string
import matplotlib.pylab as plt
from numpy import linalg as LA
%matplotlib
import seaborn as sns
import os

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

Using matplotlib backend: MacOSX
QuoraKaggle.pem
collins_duffy_test.zip
collins_duffy_train.zip
data_dump.zip
ec2-user@ec2-52-91-73-35.compute-1.amazonaws.com
sample_submission.csv
semantic_pos_features.csv
stanfordData_train.nlp
stanfordData_train_ner.nlp
test.csv
test_collins_duffy
test_preprocessed
test_raw
train.csv
train_collins_duffy
train_dean



## Extract, Transform, Load

In [10]:
from sklearn.model_selection import train_test_split

def read_data(path_to_file):
    df = pd.read_csv(path_to_file)
    print ("Shape of base training File = ", df.shape)
    print("Shape of base training data after cleaning = ", df.shape)
    return df

df_train = read_data("input/train.csv")
df_submit = read_data("input/test.csv")

# Print the column names
print (df_train.columns.values.tolist())

Shape of base training File =  (404290, 6)
Shape of base training data after cleaning =  (404290, 6)
Shape of base training File =  (2345796, 3)
Shape of base training data after cleaning =  (2345796, 3)
['id', 'qid1', 'qid2', 'question1', 'question2', 'is_duplicate']


## Load the Collins Duffy Training Features 

In [11]:
df_features = None
flip = True
base = 'input/train_collins_duffy/'
count = 0
for filename in os.listdir(base):
    if filename.endswith(".csv"): 
        tmpFrame = pd.read_csv(os.path.join(os.getcwd(), base, filename))
        print(os.path.join(os.getcwd(), base, filename))
        tmpFrame = tmpFrame.rename(columns={'cdNorm_st': filename.replace(".csv","")})
        if flip:
            df_features = tmpFrame
            flip = False
        else:
            df_features = df_features.merge(tmpFrame,how='inner',on='id')
        count+=1
        continue
    else:
        continue

/Users/Matthew/Documents/CIS700/FinalProject_Kaggle/input/train_collins_duffy/sd_1e-1_sst.csv
/Users/Matthew/Documents/CIS700/FinalProject_Kaggle/input/train_collins_duffy/sd_1e-1_st.csv
/Users/Matthew/Documents/CIS700/FinalProject_Kaggle/input/train_collins_duffy/sd_1e-2_sst.csv
/Users/Matthew/Documents/CIS700/FinalProject_Kaggle/input/train_collins_duffy/sd_1e-2_st.csv
/Users/Matthew/Documents/CIS700/FinalProject_Kaggle/input/train_collins_duffy/sd_1e0_sst.csv
/Users/Matthew/Documents/CIS700/FinalProject_Kaggle/input/train_collins_duffy/sd_1e0_st.csv
/Users/Matthew/Documents/CIS700/FinalProject_Kaggle/input/train_collins_duffy/sd_2e-1_sst.csv
/Users/Matthew/Documents/CIS700/FinalProject_Kaggle/input/train_collins_duffy/sd_2e-1_st.csv
/Users/Matthew/Documents/CIS700/FinalProject_Kaggle/input/train_collins_duffy/sd_5e-1_sst.csv
/Users/Matthew/Documents/CIS700/FinalProject_Kaggle/input/train_collins_duffy/sd_5e-1_st.csv
/Users/Matthew/Documents/CIS700/FinalProject_Kaggle/input/train_col

## Load the Collins Duffy Test Features

In [12]:
df_features_test = None
flip = True
base = 'input/test_collins_duffy/'
count = 0
for filename in os.listdir(base):
    if filename.endswith(".csv"): 
        tmpFrame = pd.read_csv(os.path.join(os.getcwd(), base, filename))
        print(os.path.join(os.getcwd(), base, filename))
        tmpFrame = tmpFrame.rename(columns={'cdNorm_st': filename.replace(".csv","")})
        if flip:
            df_features_test = tmpFrame
            flip = False
        else:
            df_features_test = df_features_test.merge(tmpFrame,how='inner',on='id')
        count+=1
        continue
    else:
        continue

/Users/Matthew/Documents/CIS700/FinalProject_Kaggle/input/test_collins_duffy/sd_1e-1_sst.csv
/Users/Matthew/Documents/CIS700/FinalProject_Kaggle/input/test_collins_duffy/sd_1e-1_st.csv
/Users/Matthew/Documents/CIS700/FinalProject_Kaggle/input/test_collins_duffy/sd_1e-2_sst.csv
/Users/Matthew/Documents/CIS700/FinalProject_Kaggle/input/test_collins_duffy/sd_1e-2_st.csv
/Users/Matthew/Documents/CIS700/FinalProject_Kaggle/input/test_collins_duffy/sd_1e0_sst.csv
/Users/Matthew/Documents/CIS700/FinalProject_Kaggle/input/test_collins_duffy/sd_1e0_st.csv
/Users/Matthew/Documents/CIS700/FinalProject_Kaggle/input/test_collins_duffy/sd_2e-1_sst.csv
/Users/Matthew/Documents/CIS700/FinalProject_Kaggle/input/test_collins_duffy/sd_2e-1_st.csv
/Users/Matthew/Documents/CIS700/FinalProject_Kaggle/input/test_collins_duffy/sd_5e-1_sst.csv
/Users/Matthew/Documents/CIS700/FinalProject_Kaggle/input/test_collins_duffy/sd_5e-1_st.csv
/Users/Matthew/Documents/CIS700/FinalProject_Kaggle/input/test_collins_duffy/

## Load in Dean's Semantic Features

In [16]:
df_semantic = pd.read_csv('input/train_dean/dean_train_features.csv')
df_semantic.drop('is_duplicate',inplace=True,axis=1)

## Training the Model

In [18]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_curve, auc, roc_curve
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import MinMaxScaler

# Read in the pre-built features for the training set
df_train = df_train.merge(df_features,how='inner',on='id')
df_train = df_train.merge(df_semantic,how='inner',on='id')

# Methodology - fill with zeroes Any row that crashed the Stanford CoreNLP parser must have been empty string
df_train.fillna(value=0, inplace=True)
    


In [19]:
df_train.columns

Index(['id', 'qid1', 'qid2', 'question1', 'question2', 'is_duplicate',
       'sd_1e-1_sst', 'sd_1e-1_st', 'sd_1e-2_sst', 'sd_1e-2_st', 'sd_1e0_sst',
       'sd_1e0_st', 'sd_2e-1_sst', 'sd_2e-1_st', 'sd_5e-1_sst', 'sd_5e-1_st',
       'sd_5e-2_sst', 'sd_5e-2_st', 'sd_8e-1_sst', 'sd_8e-1_st', 'S_sem_sim',
       'T_sem_sim', 'sem_sim', 'S_unmatch_n', 'T_unmatch_n', 'S_unmatch_n_p',
       'T_unmatch_n_p', 'S_unmatch_a', 'T_unmatch_a', 'S_unmatch_a_p',
       'T_unmatch_a_p', 'S_unmatch_v', 'T_unmatch_v', 'S_unmatch_v_p',
       'T_unmatch_v_p', 'S_unmatch_pp', 'T_unmatch_pp', 'S_unmatch_pp_p',
       'T_unmatch_pp_p', 'S_unmatch_wp', 'T_unmatch_wp', 'S_unmatch_wp_p',
       'T_unmatch_wp_p', 'S_unmatch_num', 'T_unmatch_num', 'S_unmatch_num_p',
       'T_unmatch_num_p', 'S_unmatch_ner', 'T_unmatch_ner', 'len_dif',
       'len_dif_p'],
      dtype='object')

## Adding More Features

In [20]:
# # Question Lengths
# df_train['q1len'] = df_train['question1'].str.len()
# df_train['q2len'] = df_train['question2'].str.len()

# df_submit['q1len'] = df_submit['question1'].str.len()
# df_submit['q2len'] = df_submit['question2'].str.len()

# # Normalized Word Share
# def number_of_words(row):
#     try:
#         nwords = len(row.split(" "))
#         return nwords
#     except:
#         return 0

# # Normalized Word Share
# def normalized_word_share(row):
#     try:
#         w1 = set(map(lambda word: word.lower().strip(), row['question1'].split(" ")))
#         w2 = set(map(lambda word: word.lower().strip(), row['question2'].split(" ")))    
#         return 1.0 * len(w1 & w2)/(len(w1) + len(w2))
#     except:
#         return 0

# def word_diff(row):
#     try:
#         diff = abs(row['q1_n_words']-row['q2_n_words'])
#         return diff
#     except:
#         return 0
    
# # Number of Words
# df_train['q1_n_words'] = df_train['question1'].apply(number_of_words)
# df_train['q2_n_words'] = df_train['question2'].apply(number_of_words)

# df_submit['q1_n_words'] = df_submit['question1'].apply(number_of_words)
# df_submit['q2_n_words'] = df_submit['question2'].apply(number_of_words)
    
# # Normalized Word share
# df_train['word_share'] = df_train.apply(normalized_word_share, axis=1)
# df_submit['word_share'] = df_submit.apply(normalized_word_share, axis=1)

# df_train['word_diff'] = df_train.apply(word_diff, axis=1)
# df_submit['word_diff'] = df_submit.apply(word_diff, axis=1)


## Extract the Features

In [32]:
select_cols = list(df_train.columns[6:])
df_y = df_train['is_duplicate']
df_X = df_train[select_cols].copy()

df_X.fillna(value=0, inplace=True)

In [34]:
X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.10, random_state=24)

X_train_mat = X_train.values
y_train_mat = y_train.values

X_train_mat = MinMaxScaler().fit_transform(X_train_mat)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((363853, 45), (40429, 45), (363853,), (40429,))

## Cross-Validation

In [37]:
clf = LogisticRegression()
grid = {
    'C': [1e-3, 1e0, 1e1],
    'penalty': ['l1', 'l2']
}
cv = GridSearchCV(clf, grid, scoring='neg_log_loss', n_jobs=-1, verbose=1)
cv.fit(X_train_mat, y_train_mat)

Fitting 3 folds for each of 6 candidates, totalling 18 fits


[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed: 17.3min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'C': [0.001, 1.0, 10.0], 'penalty': ['l1', 'l2']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='neg_log_loss', verbose=1)

In [38]:
# Print the best cross-validation parameters
print(cv.best_params_)
print(cv.best_estimator_.coef_)

{'C': 10.0, 'penalty': 'l2'}
[[ -7.82364154e+00   2.46798283e+00   1.04326422e+01  -8.09662493e+00
    2.86238109e+00  -3.82701394e-01  -2.22765026e+00   6.49633514e+00
   -5.56428885e-02   1.88335485e-01  -2.38147091e-01  -2.41328718e+00
   -4.15712203e+00   1.90374231e+00   6.62786067e+00   6.21046987e+00
   -4.07981770e+00  -6.44443637e+00  -1.16278961e+01  -4.95791743e-01
   -3.60776252e-01  -1.52554305e+00  -1.20942123e+00   1.86018066e-02
   -5.68118471e-02   7.30938933e-01   3.37027671e-01  -1.02553327e-01
   -1.16392352e-01  -5.41365231e+00  -4.78591848e+00   5.31513313e-01
    5.14958046e-01  -3.75284758e-01   3.08119397e-01   2.13099021e-01
    1.06026635e-02   1.68565426e-01  -2.85008043e+00  -1.34275518e-01
    3.98318803e-02  -1.42761785e+00  -2.44062226e+00   6.67954799e+00
   -1.35653666e+00]]


## ROC

In [40]:
colors = ['r', 'g', 'b', 'y', 'k', 'c', 'm', 'brown', 'r']
lw = 1
Cs = [1e-6, 1e-3, 1e0, 1e1]

plt.figure(figsize=(12,8))
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for different classifiers')

plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')

labels = []
for idx, C in enumerate(Cs):
    clf = LogisticRegression(C = C)
    clf.fit(X_train_mat, y_train_mat)
    print("C: {}, parameters {} and intercept {}".format(C, clf.coef_, clf.intercept_))
    fpr, tpr, _ = roc_curve(y_test, clf.predict_proba(X_test)[:,1])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, lw=lw, color=colors[idx])
    labels.append("C: {}, AUC = {}".format(C, np.round(roc_auc, 4)))

plt.legend(['random AUC = 0.5'] + labels)

C: 1e-06, parameters [[ -4.02694923e-03  -1.76237985e-03  -4.32947573e-03  -1.83683510e-03
   -1.16675636e-03  -2.74875372e-04  -3.62707644e-03  -1.67419565e-03
   -1.62922762e-03  -1.33967680e-03  -4.20072525e-03  -1.80413582e-03
   -1.11528606e-03  -8.42593049e-04  -1.69732036e-06  -5.76649337e-05
    8.50075313e-05  -4.93404876e-03  -2.30567130e-03  -3.63872821e-02
   -3.53036975e-02  -3.39068679e-03  -2.55325751e-03  -2.15265240e-02
   -2.14493952e-02  -4.99722007e-03  -4.52765502e-03  -3.05029229e-02
   -3.08046198e-02  -1.19343903e-03  -1.80584390e-03  -1.14720677e-02
   -1.36367353e-02  -3.00718789e-03  -2.49787586e-03  -8.75038514e-03
   -7.30146023e-03  -4.28548457e-04  -4.24322365e-04  -5.89364587e-03
   -6.33703880e-03  -1.94813454e-03  -1.82359187e-03  -1.65267204e-03
   -9.55055186e-03]] and intercept [-0.036153]
C: 0.001, parameters [[ 0.07135889  0.09322024  0.13440348  0.04339927 -0.56313248  0.23156951
  -0.0277675   0.14392116 -0.5052481   0.2697901   0.10894111  0.06

## Precision-Recall

In [119]:
pr, re, _ = precision_recall_curve(y_test, cv.best_estimator_.predict_proba(X_test)[:,1])
plt.figure(figsize=(12,8))
plt.plot(re, pr)
plt.title('PR Curve (AUC {})'.format(auc(re, pr)))
plt.xlabel('Recall')
plt.ylabel('Precision')

## Load in test set features

In [41]:
df_semantic_test = pd.read_csv('input/test_dean/dean_test_features.csv')

In [43]:
# Read in the pre-built features for the test set
df_submit = df_submit.merge(df_features_test,how='left',left_on='test_id',right_on='id')
# df_train = df_train.merge(df_semantic,how='inner',on='id')
df_submit.drop(['id'],inplace=True,axis=1)
df_submit = df_submit.merge(df_semantic_test,how='left',left_on='test_id',right_on='id')
df_submit.drop(['id'],inplace=True,axis=1)

# Methodology - fill with zeroes Any row that crashed the Stanford CoreNLP parser must have been empty string
df_submit.fillna(value=0, inplace=True)

In [47]:
# Retrain on the entire training set
retrained = cv.best_estimator_.fit(df_X.values, df_y.values)

X_test_set = df_submit[df_submit.columns[3:]].values

probs = retrained.predict_proba(df_submit[df_submit.columns[3:]].values)
df_submit['is_duplicate'] = probs[:,1]


In [49]:
# The final submission set
df_final_submit = df_submit[['test_id','is_duplicate']]

In [50]:
# plt.hist(df_submit['is_duplicate'], 50, normed=1, facecolor='green', alpha=0.75)

In [52]:
df_final_submit.to_csv("submissions/submission_v4.csv",index=False)